## Introduction:

The work from [Sathe et al. 2015](https://doi.org/10.5194/amt-8-729-2015) introduces an alternative scanning strategy to be used by wind Doppler lidars for estimating turbulence (6-beam). This new strategy consists of measuring the radial wind on five equally spaced azimuths with fixed elevation, different from 0 or 90 degrees,  and one additional measurement from the zenith.

This example shows how to use the lidarwind to retrieve wind speed and direction profiles from the 6-beam observations. The profiles are retrieved using the FFT, following the method proposed by [Ishwardat 2017](http://resolver.tudelft.nl/uuid:a659654b-e76a-4513-a656-ecad761bdbc8).

## Steps:

0) Download data from zenodo
1) Read the WindCube's data
2) Merge data from one hour of observations
3) Re-structure the data for using the wind retrieval
4) Retrieve wind speed and directin profiles
5) Visualisation

In [ ]:
import matplotlib.pyplot as plt
import lidarwind
from lidarwind.utilities import sample_data

### Step 0: Downloading the sample data

Using pooch the sample dataset will be cached in your system.

In [ ]:
file_list = sample_data('wc_6beam')


### Step 1 and 2: reading and merging the 6-beam data

In [ ]:
file_list = sorted(file_list)
merged_ds = lidarwind.DataOperations(file_list).merged_data

Below you can see all the variables available on the original WindCube's data. Not all of them are needed for retrieving the wind profiles. Note that the variables from the vertical observations are kept separate from the slanted observation. 

In [ ]:
merged_ds

First inpressions from the slanted and vertical observations

In [ ]:
merged_ds.radial_wind_speed.plot(x='time', cmap='Spectral',vmin=-1, vmax=1)
plt.show()

merged_ds.radial_wind_speed90.plot(x='time', cmap='Spectral', vmin=-1, vmax=1)
plt.show()

### Step 3: re-structuring the data

In [ ]:
restruct_data = lidarwind.GetRestructuredData(merged_ds)

The re-structured object contains all the information needed to retrieve the wind profiles. You can have a look again at the slanted and vertical observations.

In [ ]:
# Radial observation at an elevation of 75 degrees and an azimuth of 216 degrees
restruct_data.data_transf.isel(azm=3).plot(x='time',cmap='Spectral',vmin=-1, vmax=1)
plt.show()

# Observations from the zenith pointing measurements
restruct_data.data_transf_90.plot(x='time', cmap='Spectral', vmin=-1, vmax=1)
plt.show()

### Step 4: retrieving wind profiles

Wind speed and direction profiles are retrieved using the FFT method at this step. 

In [ ]:
wind_obj = lidarwind.RetriveWindFFT(restruct_data)

After retrieving the wind profiles, the dataset can be accessed by calling the wind_prop attribute. You can also visualize the retrieved wind speed and direction profiles. Have a look below.

In [ ]:
wind_obj.wind_prop

### Step 5: Visualisation

In [ ]:
wind_obj.wind_prop.horizontal_wind_speed.plot(y='range', cmap='turbo')
plt.show()

wind_obj.wind_prop.horizontal_wind_direction.plot(y='range', cmap='hsv')
plt.show()

In [ ]:
# Uncomment the line below save it as NetCDF
# wind_obj.wind_prop.to_netcdf('6_beam_example.nc')